# Testing, Mocking and Python

##### Unit testing in Python and mocking techniques.

<img src="files/unittest.jpg" height="65%" width="65%" align="left">


# Introduction

## Me
- Darshan Ahluwalia
- Using Python professionally for about 10 years
- Tutored Python at UC Irvine
- I also like [gardening](http://husbandryandharvest.tumblr.com) and making [adventure shorts](https://vimeo.com/user38627724)

<img src="files/lost-arrow.jpg" align="left">


## Workshop?
- I'll ask questions
- You ask questions
- Laptop not really needed
- Its a jupyter notebook!
- [Source on GitHub](https://github.com/darshahlu/python_testing)

<img src="files/funny-horses.jpg" align="left" height="85%" width="85%">


## Outline

1. Unit Testing Principles
   1. What
   2. Why
   3. Rules
2. Mocking
   1. Monkey Patching
   2. Template Method (Inheritance)
   3. Dependency Injection
3. Testing in Python: HOWTO
   1. unittest
   2. doctest
   3. pytest
4. Demo/example
   1. pytest demo
   2. Stock picker problem


# Unit Testing Principles
##### What, why, and rules
<img src="files/monkey-blocks.jpg" height="55%" width="55%" align="left">


##### Let's define what is unit test by first looking at the code below.

In [ ]:
def is_balanced(s):
    parens = {
        ')': '(',
        ']': '[',
        '}': '{',
    }
    opens = parens.values()
    closes = parens.keys()
    found_opens = list()
    for c in s:
        if c in opens:
            found_opens.append(c)
        elif c in closes:
            try:
                last_open = found_opens.pop()
            except IndexError:
                return False
            if parens[c] != last_open:
                return False
    return not len(found_opens)


# Do you do this? Is this testing your code? Is this Unit Test?
if __name__ == "__main__":
    print(is_balanced(''))
    print(is_balanced('hello'))


#### What is Unit Test?
- Software which tests software.
- Pieces of software, units, are tested in isolation.
- Automatable.

In [ ]:
# This is unit test!

def test_is_balanced():
    assert is_balanced('') == True
    assert is_balanced('hello') == True
    assert is_balanced('(yes)') == True
    assert is_balanced('[(nope])') == False
    assert is_balanced('{}[]([wee!])') == True
    assert is_balanced('{') == False
    assert is_balanced(')') == False


##### Now let's talk about why we write tests. What does the is_balanced function do?

#### Why Unit Test?
- Requires you to write better code: more modular, more readable, easier to identify dependencies, etc.
- Documents the code in source: tests describe behaviors and functionality.
- Supports maintainance: adding features, refactoring, etc. (Did I break anything?)
- Finds issues early: during development.
- For onboarding developers: a sanity check that the environment is setup correctly.

##### Now let's talk about unit testing rules. What are some things one must consider when writing tests?

Consider testing software which interacts with a database. What are some problems you might encounter?

<img src="files/database.png" align="left">

#### Unit Testing Rules

Rules:

1. Must require no human interaction (e.g. do not ask for inputs).
2. Must be deterministic (i.e. always produce the same result).
3. Must be fast (sub-second up to few seconds).
4. Must be testing in isolation.

Guidelines:

1. Tests should be independent from each other (no side effects).
2. Write the fewest test cases to exercise the code.

#### Unit Testing Principles Summary
- What: software which test software.
- Why: reasons. (Software quality)
- Rules: fast, isolated, and automatable.

# Mocking
### What, why, and techniques

<img src="files/mocking.jpg" align="left">



#### What is mocking 
Mocking is the changing of software behavior to specific behaviors.

#### Why
Mocking is necessary to keep tests fast, isolated and with predictable behavior.

#### Techniques
The following techniques may be employed to mock out software components:
1. monkey patch
2. template method (inheritance/override behavior)
3. dependency injection

# The Monkey Patch 
##### A mocking technique using a monkey

<img src="files/monkey-tongue-1.jpg" height="55%" width="55%" align="left">


##### To talk about patching, we need to understand: objects and typing.

In [ ]:
import math


class MyClass:
    """My class docstring"""
    A_CLASS_VAR = 1
    
    def __init__(self, a):
        self.a = a
    
    def some_method(self):
        pass

        
my_class_instance = MyClass(a='hi')


# What does this tell us about modules, classes and instances?
print('math is: ', type(math))
print('MyClass is: ', type(MyClass))
print('my_class_instance is: ', type(my_class_instance))
print()
print('math members are: ', dir(math))
print('MyClass members are: ', dir(MyClass))
print('my_class_instance members are: ', dir(my_class_instance))

In [ ]:
# Will this code execute? What does this tell us about Python's typing?
x = None
print("x is now: %s at %s" % (x, id(x)))
x = False
print("x is now: %s at %s" % (x, id(x)))
x = 1.2
print("x is now: %s at %s" % (x, id(x)))

#### In Python, everything is an object and it is dynamically typed. 


In [ ]:
# So we can do stuff like this:

class SomeClass:
    
    def hello_world(self):
        print('hello world')

s = SomeClass()



print(id(s.hello_world))
s.hello_world()



def bye_world():
    print("goodbye world")

s.hello_world = bye_world  # THE MONKEY PATCH!



print(id(s.hello_world))
s.hello_world()



What is monkey patching?  It is:

<img src="files/god-mode.jpg" align="left">

#### Definition
Dynamically change behaviors by reassigning attributes.

This can be on any object: an instance of an object, the class itself, a module or a package.

In [ ]:
import time

class Timer(object):
    
    def sleep(self, s):
        print('sleeping for %i seconds...' % s)
        time.sleep(s)
        print('done sleeping')

timer = Timer()

def expensive_operation():
    print('performing expensive thingy')
    timer.sleep(4)
    print('done with it')

# We want to test this awesome code.
def some_awesome_code(a, b, c):
    expensive_operation()
    return sum([a, b, c])


In [ ]:
# Expect 1 + 2 + 3 == 6
print(some_awesome_code(1, 2, 3))


In [ ]:
# Let's make this fast by monkey patching the timer instance attribute.

def fast_sleep(seconds):
    print('waiting %i seconds.. not!' % seconds)
    return

timer.sleep = fast_sleep


In [ ]:
some_awesome_code(1, 2, 3)


#### More About Patching
- Patching can be on any object: an instance of an object, the class itself, a module or a package.
- patching the instance
``` timer.sleep = fast_sleep ```

- patching the Timer class
``` Timer.sleep = fast_sleep ```

- patching the module directly
``` time.sleep = fast_sleep ```

#### What's the difference?


 



#### Patching Rules

- Remember unittest guidelines: no side-effects (clean up after yourself)
```
original_sleep = time.sleep
time.sleep = fast_sleep
# ... do the tests
# finally:
time.sleep = original_sleep
```
- Just avoid this and patch the instance!


# Template method
#####  A mocking technique using inheritance and donuts

<img src="files/donut-inheritance.jpg" height="55%" width="55%" align="left">


##### What is one way we normally override behavior?

#### Definition
Change behaviors by inheriting from the class-under-test and overriding behavior.

This applies to classes only.

In [ ]:
# Template method is only for classes, so refactor as such:

class MyTestableAwesomeCode(object):

    def _expensive_operation(self):
        print('performing expensive thingy')
        time.sleep(4)
        print('done with it')

    # We want to test this method
    def some_awesome_code(self, a, b, c):
        self._expensive_operation()
        return sum([a, b, c])

In [ ]:
# Expect 1 + 2 + 3 == 6
m = MyTestableAwesomeCode()
print(m.some_awesome_code(1, 2, 3))

In [ ]:
# lets make it fast by inheriting from class-under-test and overriding the slow method.

class MyTestableAwesomeCodeFast(MyTestableAwesomeCode):
    
    def _expensive_operation(self):
        print('damn fast now')

In [ ]:
m = MyTestableAwesomeCodeFast()
print(m.some_awesome_code(1, 2, 3))

##### What do you like about this method vs patching?  What do you dislike?

#### More About Classes
1. Use classes almost always for the benefit of code organization, OOP and...
2. For testability! Patch that instance or template it.
3. Multiple levels of inheritance is stinky (1-2 okay).


# Dependency Injection
##### It's like a flu shot without the side-effects

<img src="files/injection.jpg" height="50%" width="50%" align="left">


#### Definition

Fancytalk for "pass in objects as parameters" (so that you can provide objects with mocked behaviors later)

In [ ]:
# requires refactoring: make dependencies inputs

import time

def expensive_operation():
    print('performing expensive thingy')
    time.sleep(4)
    print('done with it')

def some_awesome_code(a, b, c, expensive_operation=expensive_operation):
    expensive_operation()
    return sum([a, b, c])

In [ ]:
# Expect 1 + 2 + 3 == 6
some_awesome_code(1, 2, 3)

In [ ]:
# Let's make this fast by providing a fake expensive operation
some_awesome_code(1, 2, 3, expensive_operation=lambda: None)


##### What do you like about this method over the others? Dislike?

# Patching vs Template Method vs Dependency Injection
##### Monkeys, donuts and injections
<img src="files/monkey-donut.jpg" height="60%" width="60%" align="left">

#### Patching
- Patching dynamically changes behavior by reassigning datamembers to new ones.
- Patching may be used on an object (luckily everything in Python is an object).
- Can only mock behavior that is an attribute of that object.
- Avoid patching globals like modules, packages, classes, if possible. Or restore the original behavior.

#### Template Method
- Template method statically defines new behavior (by creating a new class).
- Applicable to classes only.
- Can only mock behavior that is part of self.
- More code than patching due to typing out a new class.

#### Dependency Injection
- Fancytalk for "pass in objects as parameters"
- Requires you to provide dependencies as inputs.
- Dependencies are exposed in the interface.
- Less knowledge of class under test required.

# Testing in Python HOWTO
##### Using popular testing frameworks
<img src="files/goat-testing.jpg" align="left">


##### What frameworks are available in Python for writing unit tests?
1. Unittest
2. doctest
3. pytest
4. [and many more](https://wiki.python.org/moin/PythonTestingToolsTaxonomy)

##### 	What follows are simple examples and then those requiring setup/teardown.

# Unittest (standard library)
- [Unittest](https://docs.python.org/2/library/unittest.html) is available in Python 2 and 3 standard library
- Unittest supports test automation by writing classes.

In [ ]:
import unittest

def fib(n):
    """Return the n-th number in the fibonacci series."""
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    return fib(n-1) + fib(n-2)

class TestFib(unittest.TestCase):
    
    def test_fib_series(self):
        self.assertEqual(fib(0), 0)
        self.assertEqual(fib(1), 1)
        self.assertEqual(fib(2), 1)
        self.assertEqual(fib(3), 2)
        self.assertEqual(fib(4), 3)
        self.assertEqual(fib(5), 5)

if __name__ == "__main__":
    #normally this is unittest.main()...this is for jupyter
    unittest.main(argv=['first-arg-is-ignored'], exit=False, verbosity=2)


# Doctest (standard library)
- [Doctest](https://docs.python.org/2/library/doctest.html) is available in Python 2 and 3 standard library
- Doctest allows you to write tests in your docstrings.

In [ ]:
def fib(n):
    """Return the n-th number in the fibonacci series.
    
    >>> fib(0)
    0
    >>> fib(1)
    1
    >>> fib(29)
    514229
    """
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    return fib(n-1) + fib(n-2)


if __name__ == "__main__":
    import doctest
    #doctest.testmod()
    doctest.testmod(verbose=True)


##### What is so awesome about doctest?

# Pytest (third party)
- [Pytest](https://docs.pytest.org/en/latest/) is available for Python 2.6+ and 3.3+ as a third-party package
- Combines doctest's simplicity and unittest's advanced features into a simple and powerful framework.
- Test discovery: for doctests, unittests and pytests.


<img src="files/pytest.png" align="left">

In [ ]:
# Running pytest in jupyter requires some magic.
# See: http://cprohm.de/article/ipytest-running-pytest-in-ipython-notebooks.html
__file__ = 'Python Testing.ipynb'
import ipytest.magics
import pytest

In [ ]:
%%run_pytest[clean] -vvv

def fib(n):
    """Return the n-th number in the fibonacci series."""
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    return fib(n-1) + fib(n-2)

def test_fib_series():
    assert fib(0) == 0
    assert fib(1) == 1
    assert fib(2) == 1
    assert fib(3) == 2
    assert fib(4) == 3
    assert fib(5) == 5


##### What is great about pytest over unittest?

# Setup and teardown

Both unittest and pytest support running the same code before and after every test.


#### Unittest setup and teardown

In [ ]:
import traceback
import unittest

class SomeTestCode(unittest.TestCase):
    
    def setUp(self):
        print('patching a global module')
        self.traceback_print_tb = traceback.print_tb
        traceback.print_tb = lambda *args, **kwargs: None
        
    def tearDown(self):
        print('unpatching')
        traceback.print_tb = self.traceback_print_tb
    
    def test_one(self):
        print('in test one')
    
    def test_two(self):
        print('in test two')
       
if __name__ == "__main__":
    #normally this is unittest.main()...this is for jupyter
    unittest.main(argv=['first-arg-is-ignored'], exit=False, verbosity=0)


#### Pytest setup and teardown

Pytest provides "fixtures" for setup and teardown.


In [ ]:
%%run_pytest[clean] -qq

import pytest

@pytest.fixture(scope='function')
def mocked_module():
    print('patching')
    traceback_print_tb = traceback.print_tb
    traceback.print_tb = lambda *args, **kwargs: None
    yield
    print('unpatching')
    traceback.print_tb = traceback_print_tb

def test_one(mocked_module):
    print('in test one')
    raise Exception("boo one!")
    
def test_two(mocked_module):
    print('in test two')
    raise Exception("boo two!")

# Note that if an exception happens during the setup code (before the yield keyword),
# the teardown code (after the yield) will not be called.

# A second pattern may be employed to ensure teardown happens even when
# setup fails (use a "finalizer")


# Frameworks Summary
##### Pytest vs Unittest vs Doctest
<img src="files/fight.jpg" align="left">



#### Unittest: 
- Familiar class-based JUnit-like syntax.
- Powerful enough to do all you need.

#### Doctest: 
- Tests are both documentation and actual tests!
- Great for simple tests (e.g. functions with simple inputs and outputs).

#### Pytest
- Finds and runs all your doctests, unittests and pytests.
- Does everything unittest does but in fewer lines of code.
- Great plugins: generate coverage, get a library of django fixtures, etc.




# Pytest vs Unittest
##### Pytest wins!
<img src="files/pytestvsdoctest.png" align="left">


See also these great videos:
- [Pytest vs Unittest by Renzo Nuccitelli](https://www.youtube.com/watch?v=572xUCFCUho)
- [My Lightning Talk on Adopting Pytest](https://www.youtube.com/watch?v=sWp445cWDyA)

# Demo / Example
##### It's getting real
<br>
<img src="files/choose-your-own.jpg" align="left">


## Pytest
1. run pytests via debugger
2. run pytests in cmd line
3. generate coverage report
4. configuration: pytest.ini and .coveragerc and conftest.py

## Stock Picker Problem
Use TDD to implement a stock picker. Stock data gotten from:
- https://api.iextrading.com/1.0/stock/aapl/chart/1y
- https://api.iextrading.com/1.0/stock/aapl/price


# Stock Picker Problem

<img src="files/stock-picker.jpg" align="left">



## Task:
- build a stock picker; it identifies good stocks to buy.
- first part of stock picking is to determine if a given stock is a good buy.
- we define a stock is a good buy if its current price is less than 10% of its 1 year average price.
- price data is given by the following APIs:
  - 1 year average obtained from: https://api.iextrading.com/1.0/stock/aapl/chart/1y
  - current price: https://api.iextrading.com/1.0/stock/aapl/price

#### To make this simpler, the following template is already implemented.

In [ ]:
class StockPicker:

    def __init__(self):
        pass

    def is_good_buy(self, ticker: str) -> bool:
        """"Return True if the stock is a good buy.

        A stock is a good buy if its current price is less than 10% of its 1 year average price.
        """
        pass


##### What dependencies may require mocking out?
<br>
<br>
<br>
<br>
##### What code may introduce bugs and require special attention (testing)?
<br>
<br>
<br>
<br>
##### How may we seperate the good-buy determination from fetching data?
<br>
<br>
<br>
<br>
##### Lets start with testing the good-buy determination; what are the two simplest test cases for this method?
<br>
<br>

In [ ]:
class StockPicker:
    GOOD_BUY_THRESHOLD_PCT = 10.0

    def __init__(self):
        pass

    @classmethod
    def _is_good_buy(cls, current_price: float, one_yr_avg_price: float) -> bool:
        """Return True if `current_price` is less than a percentage of `one_yr_avg_price`.

        >>> StockPicker._is_good_buy(89.99, 100.0)
        True
        >>> StockPicker._is_good_buy(90.00, 100.0)
        False
        """
        return current_price < (one_yr_avg_price * (1 - (cls.GOOD_BUY_THRESHOLD_PCT / 100)))

    def is_good_buy(self, ticker: str) -> bool:
        """"Return True if the stock is a good buy.

        A stock is a good buy if its current price is less than 10% of its 1 year average price.
        """
        pass

if __name__ == "__main__":
    import doctest
    #doctest.testmod()
    doctest.testmod(verbose=True)


##### Now lets implement the data retrieval.


In [ ]:
import requests

class StockPricesAPI:

    URL_BASE = "https://api.iextrading.com/1.0/stock/"

    def get_current_price(self, ticker):
        url = self.URL_BASE + '{}/price'.format(ticker)
        response = requests.get(url)
        return response.json()

    def get_1_yr_price_data(self, ticker):
        url = self.URL_BASE + '{}/chart/1y'.format(ticker)
        response = requests.get(url)
        return response.json()

    def get_1_yr_avg_price(self, ticker):
        one_yr_prices = self.get_1_yr_price_data(ticker)
        average = sum([d['close'] for d in one_yr_prices]) / len(one_yr_prices)
        return average
    
if __name__ == "__main__":
    s = StockPricesAPI()
    print(s.get_current_price('aapl'))
    print(s.get_1_yr_price_data('aapl'))
    print(s.get_1_yr_avg_price('aapl'))


##### What methods are important to test in this class?
<br>
##### What issues may arise in the methods?
<br>
##### How could we test the get_1_yr_avg_price method?
<br>

In [ ]:
%%run_pytest[clean] -vvv


def test_get_1_yr_avg_price():

    def my_price_data(ticker):
        return [{'date': '2017-04-10',
                 'open': 141.3635,
                 'high': 141.6384,
                 'low': 140.6744,
                 'close': 140.9402,
                 'volume': 18933397,
                 'unadjustedVolume': 18933397,
                 'change': -0.167353,
                 'changePercent': -0.119,
                 'vwap': 141.1334,
                 'label': 'Apr 10, 17',
                 'changeOverTime': 0}]

    s = StockPricesAPI()
    s.get_1_yr_price_data = my_price_data
    assert s.get_1_yr_avg_price('some-ticker') == 140.9402

    s.get_1_yr_price_data = lambda ticker: [{'close': 3}, {'close': 1}]
    assert s.get_1_yr_avg_price('some-ticker') == 2

#### TODO: implement the is_good_buy() method and test cases.

# Conclusion
- Write them **pytests, unittests and doctests.**
- Replace behaviors to make tests fast and isolated using **patching, inheritance and injection.**
- Automate them with **jenkins** and look at pretty graphs, **coverage** and stuff.

##### program like a boss
<img src="files/like-a-boss.jpg" align="left">




# Additional Resources
- [Python Testing Tools Taxonomy](https://wiki.python.org/moin/PythonTestingToolsTaxonomy)
- [unittest](https://docs.python.org/3/library/unittest.html)
- [doctest](https://docs.python.org/3/library/doctest.html)
- [pytest](https://docs.pytest.org/en/latest/)
- [coverage](https://bitbucket.org/ned/coveragepy)
- [Learning to Test with Python](https://medium.freecodecamp.org/learning-to-test-with-python-997ace2d8abe)


# Thank You
